In [ ]:
# DreamBooth with LoRa
# Reference: https://github.com/huggingface/diffusers

In [ ]:
# !git clone https://github.com/huggingface/diffusers
# updated train_dreambooth_lora in diffusers/dreambooth to remove logging_dir due to error
!git clone https://github.com/kdmytryshyn/customDiffusionCrossAttention.git


Cloning into 'customDiffusionCrossAttention'...
remote: Enumerating objects: 925, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 925 (delta 17), reused 0 (delta 0), pack-reused 896
Receiving objects: 100% (925/925), 181.24 MiB | 27.52 MiB/s, done.
Resolving deltas: 100% (375/375), done.
Updating files: 100% (798/798), done.


# New Section

In [ ]:
%cd diffusers
!pip install -e .

/content/diffusers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-0.editable-py3-none-any.whl size=10518 sha256=ffae74061f9aa77a51cede71e332d30c7489cab3a69f885a5c331a679c35fa16
  Stored in directory: /tmp/pip-ephem-wheel-cache-818miqbf/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers


In [ ]:
# !pip install git+https://github.com/huggingface/accelerate

In [ ]:
%cd examples/dreambooth

/content/diffusers/examples/dreambooth


In [ ]:
!pip install -r requirements.txt
#this installs accelerate (our environment) as well as:
# accelerate>=0.16.0, torchvision, transformers>=4.25.1, ftfy, tensorboard, Jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 123.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


False

In [ ]:
#https://colab.research.google.com/drive/1U7SX7jNYsNQG5BY1xEQQHu48Pn6Vgnyt?usp=sharing#scrollTo=O_A9ss7egPp4
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
# Now let's get our dataset. For this example we will use some dog images: https://huggingface.co/datasets/diffusers/dog-example.
# Let's first download it locally:
# note this creates a 'dog' folder in the folder we are currently in (see LHS folders on this page)
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/diffusers/examples/dreambooth/dog'

In [ ]:
!pip install wandb -qU

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# # LESS GPU - BITS AND BYTES
# !pip install bitsandbytes
# !pip install -U xformers

In [ ]:
run = wandb.init(
    # Set the project where this run will be logged
    project="lora-db",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.0001,
        "epochs": 10,
    })

wandb: Currently logged in as: kdmy (cs230). Use `wandb login --relogin` to force relogin


In [ ]:
MODEL_NAME="runwayml/stable-diffusion-v1-5"
INSTANCE_DIR="dog"
OUTPUT_DIR="kristend/lora_db"

!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=50 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=10 \
  --validation_epochs=10 \
  --seed="0" \
  --report_to="wandb" \
  --push_to_hub

# --train_text_encoder \

2023-06-07 20:39:14.610608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-07 20:39:19.245119: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
06/07/2023 20:39:21 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'dynamic_thresholding_ratio', 'sample_max_value', 'thresholding', 'prediction_type', 'variance_type', 'clip_sample_range'} was not found in config. Values will be initialized to default values.
{'scaling_factor'} was not found in config. Values will be initialized to default values.
{'time_embedding_type', 'mid_block_type', 'addition_embed_type', 'addition_embed_type_num_heads', 'use_linear_projection', 'resnet_skip_time_act',

In [ ]:
# # After training, LoRA weights can be loaded very easily into the original pipeline. First, you need to load the original pipeline
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
# # import torch
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# # Next, we can load the adapter layers into the UNet with the [`load_attn_procs` function]
# # (https://huggingface.co/docs/diffusers/api/loaders#diffusers.loaders.UNet2DConditionLoadersMixin.load_attn_procs).

pipe.load_attn_procs("kristend/lora_dba")

# Finally, we can run the model in inference.

image = pipe("A picture of a sks dog in a bucket", num_inference_steps=25).images[0]